##### *Copyright 2021 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Compile a model for the Edge TPU

This notebook offers a convenient way to compile a TensorFlow Lite model for the Edge TPU, in case you don't have a system that's compatible with the [Edge TPU Compiler](https://coral.ai/docs/edgetpu/compiler/) (Debian Linux only).

Simply upload a compatible `.tflite` file to this Colab session, run the code below, and then download the compiled model.

For more details about how to create a model that's compatible with the Edge TPU, see the [documentation at coral.ai](https://coral.ai/docs/edgetpu/models-intro/).


<a href="https://colab.research.google.com/github/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


## Uploading the compatible TF Lite models

The TensorFlow Lite model need to be fully quantized and need to meet all the [Edge TPU model requirements](https://coral.ai/docs/edgetpu/models-intro/#model-requirements).

I have fully quantized the models and stored them on sciebo where I will download them from.

In [ ]:
! mkdir raw_models

In [ ]:
! wget -O raw_models/densenet_121_224_quant.tflite https://rwth-aachen.sciebo.de/s/yOAxPVFQR8pIk2K/download

In [ ]:
! wget -O raw_models/inception_v3_299_quant.tflite https://rwth-aachen.sciebo.de/s/TuEhOkNvjvlSQEd/download

In [ ]:
! wget -O raw_models/mobilenet_v2_1.0_224_quant.tflite https://rwth-aachen.sciebo.de/s/KpU0yOOFstur4GQ/download

In [ ]:
! wget -O raw_models/resnet_50_224_quant.tflite https://rwth-aachen.sciebo.de/s/SVPNGYU5sKf50r5/download

In [ ]:
! wget -O raw_models/vgg16_224_quant.tflite https://rwth-aachen.sciebo.de/s/Z5wpaFkchaA1Dee/download

Checking the file sizes if they look right:

In [ ]:
! ls -lh raw_models/

## Get the Edge TPU Compiler

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

## Compiling all combinations of the models...

Before we start, let's take care of the imports.

In [ ]:
import os
from itertools import chain, combinations

First, I create a mapping of the models so it's easier to run a script. Adding the folder to the name makes it easier for running the commands later.

In [ ]:
model_folder_mapping = { 0: 'raw_models/densenet_121_224_quant.tflite',
                   1: 'raw_models/inception_v3_299_quant.tflite',
                   2: 'raw_models/mobilenet_v2_1.0_224_quant.tflite',
                   3: 'raw_models/resnet_50_224_quant.tflite',
                   4: 'raw_models/vgg16_224_quant.tflite'}

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    # returning powerset starting at combinations of 2 onwards
    return chain.from_iterable(combinations(s, r) for r in range(2, len(s)+1))

In [ ]:
model_combinations = list(powerset(model_folder_mapping.keys()))

In [ ]:
model_combinations

Creating the directory to save the co-compiled models in different subdirectories.

In [ ]:
! mkdir compiled_models

In [ ]:
for i, comb in enumerate(model_combinations):
  print('COMPILING NR {}...'.format(i))
  cur_dir = 'compiled_models/models_' + str(i)
  compile_command = '/usr/bin/edgetpu_compiler -o ' + cur_dir
  os.makedirs(cur_dir, exist_ok=True)
  for model in comb:
    compile_command = compile_command + ' ' + model_folder_mapping[model]
  os.system(compile_command)
  print("Compiling {} done! Next one...".format(i))

print("Compiling finished.")

Now the models are all co-compiled and in a folder with the other co-compiled ones. There is one folder for each combination. I just named iteratively from models_0 to models_25. Check folders to see which ones are in there.
We also need the `inference_reference.csv` files to run our script on the Edge TPU. Therefore, we add the csv file to each folder. We also need the benchmark time for the models, so I'm creating another mapping for the times. For Inception v3 is no benchmark time given, so I'm using benchmark time from Inception v4. Also DenseNet is not further specified, so again I'm using the one from the [docs](https://coral.ai/docs/edgetpu/benchmarks/).

In [ ]:
time_mapping = {
    0: 25,
    1: 102,
    2: 2.6,
    3: 56,
    4: 343
}
# another model_mapping with 'edgetpu' in the name, so the compiled models
model_mapping = { 0: 'densenet_121_224_quant_edgetpu.tflite',
                   1: 'inception_v3_299_quant_edgetpu.tflite',
                   2: 'mobilenet_v2_1.0_224_quant_edgetpu.tflite',
                   3: 'resnet_50_224_quant_edgetpu.tflite',
                   4: 'vgg16_224_quant_edgetpu.tflite'}

In [ ]:
for i, comb in enumerate(model_combinations):
  cur_dir = 'compiled_models/models_' + str(i)
  create_csv_cmd = 'touch ' + cur_dir + '/inference_reference_aarch64.csv'
  os.system(create_csv_cmd)
  insert_header_cmd = 'echo "MODEL INFERENCE_TIME" >> ' + cur_dir + '/inference_reference_aarch64.csv' 
  os.system(insert_header_cmd)
  for model in comb:
    model_with_time = model_mapping[model] + ' ' + str(time_mapping[model])
    insert_model_and_time = 'echo ' + model_with_time + ' >> ' + cur_dir + '/inference_reference_aarch64.csv' 
    os.system(insert_model_and_time)

## Download the model

Zipping the folder with the compiled files and downloading it.

In [ ]:
!zip -r /content/compiled_models.zip /content/compiled_models

## Run the model on the Edge TPU




You can now run the model on your Coral device with acceleration on the Edge TPU.

Check out some examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).